##### RAG task in both frameworks and compare to compare code

In [11]:
# Import libraries
import PyPDF2
from dotenv import load_dotenv
import os

# Custom RAG (simplified from module 6)
from sentence_transformers import SentenceTransformer
import numpy as np
from openai import OpenAI

In [35]:
# Load the Api key
load_dotenv()
api_key = os.getenv("test_api")

if not openai_api_key:
    print("Warning: TEST_API NOT FOUND. Set it in .env file")
else:
    print("API key loaded successfully")

API key loaded successfully


In [36]:
# Create test sentences
documents = [
    "Python is a high-level programming language known for readability and simplicity",
    "Machine learning is a subset of AI that enables systems to learn from data.",
    "RAG combines retrievalo and generation to provide accurate, grounded responses.",
    "Python functions are blocks of reusable code.",
    "A function in Python allows code reuse.",
    "Dogs are loyal animals."
]

In [37]:
# Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(documents)

'(ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)), '(Request ID: 0548cf77-3a84-4366-b746-e4d731fd4c20)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [38]:
# Query and retrieve
query = "What is RAG?"
query_embedding = model.encode([query])[0]

In [39]:
# compute similarity
similarities = np.dot(doc_embeddings, query_embedding)
top_idx = np.argmax(similarities)
retrieved_doc = documents[top_idx]

retrieved_doc

'RAG combines retrievalo and generation to provide accurate, grounded responses.'

In [40]:
# Generate response
client = OpenAI(api_key=api_key)
prompt = f"""Context: {retrieved_doc}

Question: {query}

Answer based on the context"""
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages= [{"role": "user", "content": prompt}],
    temperature=0
)

print("Custom RAG Answer")
print(response.choices[0].message.content)

Custom RAG Answer
RAG is a system that combines retrieval and generation to provide accurate, grounded responses.


In [42]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Convert document
lc_doc = [Document(page_content=doc) for doc in documents]

# Vector store
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = FAISS.from_documents(lc_doc, embeddings)
retriever = vectorstore.as_retriever()

# LLM
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
    openai_api_key= api_key
)

# Prompt
prompt = ChatPromptTemplate.from_messages([
("system", "You are an expert assistant. Use ONLY the retrieved context"),
("human", "{question}\n\n Context:\n{context}")
])

# Build RAG pipeline
rag_chain = (RunnableParallel(context=retriever, question=RunnablePassthrough())|prompt|llm)

# Query
response = rag_chain.invoke("What is RAG?")
print(response)

content='RAG stands for Retrieval-Augmented Generation. It combines retrieval and generation to provide accurate, grounded responses.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 209, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-ClCETtbFchY0Dv5DKvh9EgoohOZpe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b07ec-2225-7fb0-b892-ba3adbb2b6e3-0' usage_metadata={'input_tokens': 209, 'output_tokens': 21, 'total_tokens': 230, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
# # Sample documents for all examples
# # Load the PDF
# file_path = r"C:\Users\kanyi\OneDrive\Desktop\module_4\RAG\Python For Dummies.pdf"
# file_name = os.path.basename(file_path)
# full_text = ""

# # Open the PDF
# with open(file_path, 'rb') as f:
#     pdf_reader = PyPDF2.PdfReader(f)

#     # Process each page
#     for page in pdf_reader.pages:
#         # Extract text from page
#         text = page.extract_text()
#         if text:
#         # Extract full text
#             full_text += text + "\n"
#             print(full_text)
